In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = ""
os.environ["GEMINI_API_KEY"] = ""

In [2]:


import os
import requests
import json
from typing import Any, List, Optional
from langchain_core.prompts import PromptTemplate
from langchain_core.language_models.llms import BaseLLM
from langchain_core.outputs import LLMResult

from pydantic import BaseModel, Field, validator
from langchain_core.output_parsers import StrOutputParser

from langchain_core.runnables import RunnableLambda # Added for mapping chain outputs


In [3]:

# --- 1. Custom LLM Implementation ---

class CustomHTTPGemini(BaseLLM):
    """
    A custom LangChain LLM wrapper that interacts with the Google Gemini API
    using direct HTTP requests (POST to generateContent endpoint).
    """

    # Model and API Configuration
    api_key: Optional[str] = None
    model_name: str = Field(default="gemini-2.5-flash", alias="model")
    # Base URL remains for configuration, though we construct the full endpoint in _call
    base_url: str = "https://generativelanguage.googleapis.com/v1beta/models/"

    def __init__(self, **kwargs: Any):
        super().__init__(**kwargs)
        # Ensure the API key is set, prioritizing the passed argument or environment variable
        if not self.api_key:
            self.api_key = os.getenv("GEMINI_API_KEY")

        if not self.api_key:
            raise ValueError("GEMINI_API_KEY must be provided or set as an environment variable.")

    @property
    def _llm_type(self) -> str:
        """Return type of LLM."""
        return "custom_http_gemini"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[Any] = None,
        **kwargs: Any,
    ) -> str:
        """
        The core logic to make the HTTP POST request to the Gemini API.

        This method is called by the LangChain framework when the LLM is invoked.
        """
        # 1. Construct the API Endpoint for the specific model and method
        # This now explicitly defines the full endpoint structure similar to the reference code.
        api_endpoint = f"{self.base_url}{self.model_name}:generateContent"

        # 2. Construct the complete URL with API Key as query parameter
        url = f"{api_endpoint}?key={self.api_key}"

        # 3. Define the HTTP headers
        headers = {
            "Content-Type": "application/json"
        }

        # 4. Construct the JSON request body following the Gemini API spec
        request_data = {
            "contents": [
                {
                    "parts": [
                        {
                            "text": prompt
                        }
                    ]
                }
            ]
            # Optional: Add generation config parameters if needed (e.g., temperature, max_output_tokens)
            # "generationConfig": {
            #     "temperature": 0.7
            # }
        }

        # 5. Send the request
        try:
            # Using 'json=request_data' is a cleaner way to send JSON data with requests
            response = requests.post(
                url=url,
                headers=headers,
                json=request_data
            )
            response.raise_for_status() # Raise exception for bad status codes

            response_json = response.json()

            # 6. Extract the generated text from the structured JSON response
            # Path: candidates[0].content.parts[0].text
            generated_text = response_json['candidates'][0]['content']['parts'][0]['text']

            return generated_text

        except requests.exceptions.HTTPError as err:
            error_message = f"Gemini API HTTP Error ({err.response.status_code}): {err.response.text}"
            raise RuntimeError(error_message) from err
        except Exception as e:
            raise RuntimeError(f"An unexpected error occurred during API call: {e}")

    # Note: _generate is required by BaseLLM if _call is not implemented, but since
    # we implemented _call for simplicity, we provide a basic _generate for completeness
    # in case of future changes in the base class.
    def _generate(
        self,
        prompts: List[str],
        stop: Optional[List[str]] = None,
        run_manager: Optional[Any] = None,
        **kwargs: Any,
    ) -> LLMResult:
        """Call the LLM on a list of prompts."""
        generations = []
        for prompt in prompts:
            text = self._call(prompt, stop, run_manager, **kwargs)
            generations.append([{"text": text}]) # Wrap the result in the expected structure
        return LLMResult(generations=generations)



In [4]:

# --- 2. LCEL Chain Integration (Cascading Agents) ---

if __name__ == "__main__":
    # --- Setup ---
    print("--- LangChain Custom HTTP Gemini Cascading Agent Example ---")

    # NOTE: Set your API Key in your environment before running:
    # export GEMINI_API_KEY="YOUR_API_KEY_HERE"

    # Initialize the custom LLM
    try:
        # custom_llm will automatically pick up the API key from the environment variable
        custom_llm = CustomHTTPGemini(model_name="gemini-2.5-flash")
    except ValueError as e:
        print(f"\nERROR: {e}")
        print("Please set the GEMINI_API_KEY environment variable and try again.")
        exit()

    # --- AGENT 1: Summarizer Chain (takes 'topic' as input) ---

    prompt_1 = PromptTemplate.from_template(
        "Summarize the following concept into three key, concise bullet points, ensuring the output is easy to understand: {topic}"
    )

    # Chain 1: Summarizer (outputs a string containing the summary)
    chain_1 = prompt_1 | custom_llm | StrOutputParser()

    # --- Mapping Step ---
    # The output of Chain 1 is a string. Chain 2 needs a dictionary input {'summary': string}.
    # This RunnableLambda maps the string output to the key expected by Chain 2.
    def map_summary_to_context(summary_text: str) -> dict:
        return {"summary": summary_text}

    summary_mapper = RunnableLambda(map_summary_to_context)


    # --- AGENT 2: Tweet Writer Chain (takes 'summary' as input) ---

    prompt_2 = PromptTemplate.from_template(
        "Based on the summary below, write a creative and engaging tweet (max 280 characters) for a science blog. Include relevant hashtags and emojis.\n\nSUMMARY:\n{summary}"
    )

    # Chain 2: Tweet Writer (outputs a string containing the final tweet)
    chain_2 = prompt_2 | custom_llm | StrOutputParser()

    # --- FINAL CASCADING CHAIN ---
    # Chain 1 -> Summary Mapper -> Chain 2
    final_chain = chain_1 | summary_mapper | chain_2

    # --- Example Invocation ---
    print("\nInvoking the two-agent cascading chain: Summarizer -> Tweet Writer...")
    topic_to_explain = "The concept of cold fusion and why it is controversial"

    # The final chain only requires the input for the first chain's prompt ({topic})
    response = final_chain.invoke({"topic": topic_to_explain})

    print(f"\nOriginal Topic: {topic_to_explain}")
    print("\n--- Final Output (Generated by Chain 2 based on Chain 1's summary) ---")
    print(response)
    print("\n--- End of Cascading Chain Execution ---")

--- LangChain Custom HTTP Gemini Cascading Agent Example ---

Invoking the two-agent cascading chain: Summarizer -> Tweet Writer...

Original Topic: The concept of cold fusion and why it is controversial

--- Final Output (Generated by Chain 2 based on Chain 1's summary) ---
Remember cold fusion? Imagine clean nuclear energy at room temp! 🤩 Sounds amazing, right? But despite 1989 hype, inconsistent replication & lack of theory meant mainstream science largely cooled on it. A classic science controversy! #ColdFusionDebate #ScienceFacts #NuclearEnergy ⚛️

--- End of Cascading Chain Execution ---
